In [10]:
import os
import re
import pandas as pd
from datetime import timedelta

os.chdir('C:\\Users\\15714\\Documents\\repositories\\publicsandbox\\fishing_report_dashboard')
pd.set_option('display.max_columns', None)

In [11]:
from datetime import datetime, date

from get_data.weather_data import get_historical_weather
from get_data.water_data import get_usgs_water_data
from get_data.sun_data import get_sun_data


# Roanoke site
site = '02055080'

# Roanoke home launch
lat = 37.25511924499429
lon = -79.94058843602322

# USGS parameter code
param_cd = '00010'

# start/end
start_dt = datetime(2023, 1, 1)
end_dt = datetime(2023, 3, 1)

usgs_water_data = get_usgs_water_data(
    site,
    param_cd,
    start_dt.strftime('%Y-%m-%dT%H:%M:%S-05:00'),
    end_dt.strftime('%Y-%m-%dT%H:%M:%S-05:00')
)
historical_weather = get_historical_weather(lat, lon, start_dt, end_dt)

sun_data = get_sun_data(lat, lon, start_dt, end_dt)

def filter_hour_minute(df, col, hour, minute):    
    df = df[df[col].apply(lambda x: x.hour) == hour]
    df = df[df[col].apply(lambda x: x.minute) == minute]
    return df

In [12]:
usgs_water_data

,agency_cd,site_no,datetime,tz_cd,water_temp
0,USGS,2055080,2023-01-01 01:00:00,EST,45.14
1,USGS,2055080,2023-01-01 01:15:00,EST,45.14
2,USGS,2055080,2023-01-01 01:30:00,EST,45.14
3,USGS,2055080,2023-01-01 01:45:00,EST,45.14
4,USGS,2055080,2023-01-01 02:00:00,EST,45.14
...,...,...,...,...,...
5344,USGS,2055080,2023-03-01 00:00:00,EST,53.96
5345,USGS,2055080,2023-03-01 00:15:00,EST,53.78
5346,USGS,2055080,2023-03-01 00:30:00,EST,53.78
5347,USGS,2055080,2023-03-01 00:45:00,EST,53.78


In [13]:
historical_weather.head(5)

,air_temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,datetime,Weather Condition
0,51.98,10.6,97.0,0.0,NaN,50.0,5.4,NaN,1012.9,NaN,2023-01-01 00:00:00,Fog
1,48.92,8.3,93.0,0.0,NaN,0.0,0.0,NaN,1013.8,NaN,2023-01-01 01:00:00,Fog
2,51.98,10.0,93.0,0.0,NaN,260.0,9.4,NaN,1014.3,NaN,2023-01-01 02:00:00,Fog
3,51.98,10.0,93.0,0.0,NaN,260.0,9.4,NaN,1014.3,NaN,2023-01-01 03:00:00,Fog
4,42.98,5.1,93.0,0.0,NaN,160.0,11.2,NaN,1014.1,NaN,2023-01-19 13:00:00,Fog


In [14]:
sun_data.head(5)

,sundata_lat,sundata_lon,date,sunrise,sunset,sun_hours
0,37.255119,-79.940588,2023-01-01,2023-01-01 07:34:00,2023-01-01 17:13:00,9.650000
1,37.255119,-79.940588,2023-01-02,2023-01-02 07:34:00,2023-01-02 17:14:00,9.666667
2,37.255119,-79.940588,2023-01-03,2023-01-03 07:34:00,2023-01-03 17:15:00,9.683333
3,37.255119,-79.940588,2023-01-04,2023-01-04 07:34:00,2023-01-04 17:15:00,9.683333
4,37.255119,-79.940588,2023-01-05,2023-01-05 07:34:00,2023-01-05 17:16:00,9.700000


In [15]:
print(sun_data.dtypes)
print(historical_weather.dtypes)
print(usgs_water_data.dtypes)

sundata_lat           float64
sundata_lon           float64
date                   object
sunrise        datetime64[ns]
sunset         datetime64[ns]
sun_hours             float64
dtype: object
air_temp                    float64
dwpt                        float64
rhum                        float64
prcp                        float64
snow                        float64
wdir                        float64
wspd                        float64
wpgt                        float64
pres                        float64
tsun                        float64
datetime             datetime64[ns]
Weather Condition            object
dtype: object
agency_cd             object
site_no                int64
datetime      datetime64[ns]
tz_cd                 object
water_temp           float64
dtype: object


In [16]:
def add_join_keys(df, rnm_sfx, timecol = 'datetime', minute = False):
    if minute:
        df['_join_minute'] = df[timecol].apply(lambda x: x.minute)
    df['_join_hour'] = df[timecol].apply(lambda x: x.hour)
    df['_join_day'] = df[timecol].apply(lambda x: x.day)
    df['_join_month'] = df[timecol].apply(lambda x: x.month)
    df['_join_year'] = df[timecol].apply(lambda x: x.year)
    
    df = df.rename(columns = {timecol: f'{timecol}_{rnm_sfx}'})
    
    return df


def drop_join_keys(df):
    for i in ['_join_minute', '_join_hour', '_join_day', '_join_month', '_join_year']:
        try:
            df = df.drop(i, axis=1)
        except:
            print(f'{i} not found in DataFrame\n')
        finally:
            pass
    
    return df

In [17]:
usgs_water_data = add_join_keys(usgs_water_data, 'water')
historical_weather = add_join_keys(historical_weather, 'weather')

# future note: I push the hour for weather up 1, so that the join gets water temperature's weather at either 1 hour before,
# 1:15 before, 1:30 before, or 1:45 before
historical_weather['_join_hour'] = historical_weather['_join_hour'].apply(lambda x: (x + 1) % 24)

merged = pd.merge(
    usgs_water_data,
    historical_weather,
    on = ['_join_hour', '_join_day', '_join_month', '_join_year']
)

In [18]:
drop_join_keys(merged)

_join_minute not found in DataFrame



,agency_cd,site_no,datetime_water,tz_cd,water_temp,air_temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,datetime_weather,Weather Condition
0,USGS,2055080,2023-01-01 01:00:00,EST,45.14,51.98,10.6,97.0,0.0,NaN,50.0,5.4,NaN,1012.9,NaN,2023-01-01 00:00:00,Fog
1,USGS,2055080,2023-01-01 01:15:00,EST,45.14,51.98,10.6,97.0,0.0,NaN,50.0,5.4,NaN,1012.9,NaN,2023-01-01 00:00:00,Fog
2,USGS,2055080,2023-01-01 01:30:00,EST,45.14,51.98,10.6,97.0,0.0,NaN,50.0,5.4,NaN,1012.9,NaN,2023-01-01 00:00:00,Fog
3,USGS,2055080,2023-01-01 01:45:00,EST,45.14,51.98,10.6,97.0,0.0,NaN,50.0,5.4,NaN,1012.9,NaN,2023-01-01 00:00:00,Fog
4,USGS,2055080,2023-01-01 02:00:00,EST,45.14,48.92,8.3,93.0,0.0,NaN,0.0,0.0,NaN,1013.8,NaN,2023-01-01 01:00:00,Fog
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5340,USGS,2055080,2023-02-28 23:00:00,EST,53.96,66.02,-3.3,22.0,0.0,NaN,350.0,18.4,NaN,1010.8,NaN,2023-02-28 22:00:00,Clear
5341,USGS,2055080,2023-02-28 23:15:00,EST,53.96,66.02,-3.3,22.0,0.0,NaN,350.0,18.4,NaN,1010.8,NaN,2023-02-28 22:00:00,Clear
5342,USGS,2055080,2023-02-28 23:30:00,EST,53.96,66.02,-3.3,22.0,0.0,NaN,350.0,18.4,NaN,1010.8,NaN,2023-02-28 22:00:00,Clear
5343,USGS,2055080,2023-02-28 23:45:00,EST,53.96,66.02,-3.3,22.0,0.0,NaN,350.0,18.4,NaN,1010.8,NaN,2023-02-28 22:00:00,Clear
